In [ ]:
pip install -U stellar-sdk

In [2]:
from stellar_sdk import Keypair

In [3]:
pair = Keypair.random() # creating a random keypair

print(f"Secret: {pair.secret}") #private_key
print(f"Public Key: {pair.public_key}") #public_key

Secret: SDJCFP25BRZYGK76BIKQ4TFC4ECBT5ACFE3DXEJ57VB7K3WRMJFAZBNZ
Public Key: GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ


In [4]:
import requests

public_key = str(pair.public_key)

#In Stellar keypairs can exsist but an acoount can only be created from a public_key if that public_key has a balance of >= 1 XLM
#But since we are in test network, we can request free XLM and create an account from "Freindbot"
# AccountId = Publickey(used to request)
response = requests.get(f"https://friendbot.stellar.org?addr={public_key}")

if response.status_code == 200:
  print(f"SUCCESS! You have a new account :)\n{response.text}")
else:
  print(f"ERROR! Response: \n{response.text}")

SUCCESS! You have a new account :)
{
  "_links": {
    "self": {
      "href": "https://horizon-testnet.stellar.org/transactions/b065aa51e0f412667bc2347c145046be0e0680dff419682ce5ba8d5b3b74d78f"
    },
    "account": {
      "href": "https://horizon-testnet.stellar.org/accounts/GAN56N7BQTUSLSXP662XTWL3PMNSRLEF2TGP67HAWREXU6MAQR4TNVVP"
    },
    "ledger": {
      "href": "https://horizon-testnet.stellar.org/ledgers/25238"
    },
    "operations": {
      "href": "https://horizon-testnet.stellar.org/transactions/b065aa51e0f412667bc2347c145046be0e0680dff419682ce5ba8d5b3b74d78f/operations{?cursor,limit,order}",
      "templated": true
    },
    "effects": {
      "href": "https://horizon-testnet.stellar.org/transactions/b065aa51e0f412667bc2347c145046be0e0680dff419682ce5ba8d5b3b74d78f/effects{?cursor,limit,order}",
      "templated": true
    },
    "precedes": {
      "href": "https://horizon-testnet.stellar.org/transactions?order=asc\u0026cursor=108396384620544"
    },
    "succeeds": {

In [5]:
from stellar_sdk import Server

server = Server("https://horizon-testnet.stellar.org") #Stellar-Testnet server

# Now fetching the account from the testnet server
account = server.accounts().account_id(public_key).call()

# Now fetching the 'balance' of that account
for balance in account['balances']:
  print(f"Type: {balance['asset_type']}, Balance: {balance['balance']}")


Type: native, Balance: 10000.0000000


In [6]:
account

{'_links': {'data': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ/data/{key}',
   'templated': True},
  'effects': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ/effects{?cursor,limit,order}',
   'templated': True},
  'offers': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ/offers{?cursor,limit,order}',
   'templated': True},
  'operations': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ/operations{?cursor,limit,order}',
   'templated': True},
  'payments': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ/payments{?cursor,limit,order}',
   'templated': True},
  'self': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPN

In [9]:
##Creating a second acoount

pair2 = Keypair.random()

public_key2 = str(pair2.public_key)

#In Stellar keypairs can exsist but an acoount can only be created from a public_key if that public_key has a balance of >= 1 XLM
#But since we are in test network, we can request free XLM and create an account from "Freindbot"
# AccountId = Publickey(used to request)
response = requests.get(f"https://friendbot.stellar.org?addr={public_key2}")

if response.status_code == 200:
  print(f"SUCCESS! You have a new account :")
else:
  print(f"ERROR! Response:")



SUCCESS! You have a new account :


In [11]:
## Send a Payment
from stellar_sdk import Asset, Keypair, Network, Server, TransactionBuilder
from stellar_sdk.exceptions import NotFoundError, BadResponseError, BadRequestError

server = Server("https://horizon-testnet.stellar.org") #Testnet-server


# First, check to make sure that the destination account exists.
# You could skip this, but if the account does not exist, you will be charged
# the transaction fee when the transaction fails.
try:
    server.load_account(public_key2)
except NotFoundError:
    # If the account is not found, surface an error message for logging.
    raise Exception("The destination account does not exist!")


# loading our account
source_account = server.load_account(pair.public_key)

#fetching transaction fee from the network
base_fee = server.fetch_base_fee()


#start building the transaction
transaction = (
    TransactionBuilder(
        source_account= source_account,
        network_passphrase = Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee = base_fee,
    )

        # Because Stellar allows transaction in many currencies, you must specify the asset type.
        # Here we are sending Lumens.
        .append_payment_op(destination = public_key2, asset = Asset.native(), amount = '10')

        # A memo allows you to add your own metadata to a transaction. It's
        # optional and does not affect how Stellar treats the transaction.
        .add_text_memo("Test Transaction")

        # Wait a maximum of three minutes for the transaction
        .set_timeout(10)
        .build()
)


# Sign the transaction to prove you are actually the person sending it.
transaction.sign(pair.secret)


try:
  # And finally, send it off to Stellar!
  response = server.submit_transaction(transaction)
  print(f"Response: {response}")
except (BadRequestError, BadResponseError) as err:
  print(f"Something went wrong! \n{err}")



Response: {'memo': 'Test Transaction', 'memo_bytes': 'VGVzdCBUcmFuc2FjdGlvbg==', '_links': {'self': {'href': 'https://horizon-testnet.stellar.org/transactions/33905b659b8acddf798e787c935f84eebdf85cd3dd4279fc85f2178d7c1c3842'}, 'account': {'href': 'https://horizon-testnet.stellar.org/accounts/GAVR5MIFILZECQCGBXUWBMLTWRHOKH5UBC2ANXK74ZWPNSKK4RIMNNVJ'}, 'ledger': {'href': 'https://horizon-testnet.stellar.org/ledgers/26543'}, 'operations': {'href': 'https://horizon-testnet.stellar.org/transactions/33905b659b8acddf798e787c935f84eebdf85cd3dd4279fc85f2178d7c1c3842/operations{?cursor,limit,order}', 'templated': True}, 'effects': {'href': 'https://horizon-testnet.stellar.org/transactions/33905b659b8acddf798e787c935f84eebdf85cd3dd4279fc85f2178d7c1c3842/effects{?cursor,limit,order}', 'templated': True}, 'precedes': {'href': 'https://horizon-testnet.stellar.org/transactions?order=asc&cursor=114001316941824'}, 'succeeds': {'href': 'https://horizon-testnet.stellar.org/transactions?order=desc&cursor=

In [ ]:
## A simple program that watches the network for payments 

def load_last_paging_token():
    # Get the last paging token from a local database or file
    return "now"

def save_paging_token(paging_token):
    # In most cases, you should save this to a local database or file so that
    # you can load it next time you stream new payments.
    pass


# Create an API call to query payments involving the account.
payments = server.payments().for_account(public_key)

# There are two main parts to this program. First, you create a query for 
# payments involving a given account. Like most queries in Stellar, this could return a
# huge number of items, so the API returns paging tokens, which you can use later 
# to start your query from the same point where you previously left off. 
# In the example above, the functions to save and load paging tokens are left blank, 
# but in a real application, you’d want to save the paging tokens to a file or database so you 
# can pick up where you left off in case the program crashes or the user closes it.

last_token = load_last_paging_token() #getting the bookmark
if last_token:
  payments.cursor(last_token) #cursor to the bookmark



# `stream` will send each recorded payment, one by one, then keep the
# connection open and continue to send you new payments as they occur.
for payment in payments.stream():
   # Record the paging token so we can start from here next time.
    save_paging_token(payment["paging_token"])

    # We only process `payment`, ignore `create_account` and `account_merge`.
    if payment["type"] != "payment":
        continue

    # The payments stream includes both sent and received payments. We
    # only want to process received payments here.
    if payment['to'] != public_key:
        continue

    # In Stellar’s API, Lumens are referred to as the “native” type. Other
    # asset types have more detailed information.
    if payment["asset_type"] == "native":
        asset = "Lumens"
    else:
        asset = f"{payment['asset_code']}:{payment['asset_issuer']}"

    print(f"{payment['amount']} {asset} from {payment['from']}")
